In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import datetime
import os

In [4]:
import Funtion_tools as Ft

In [52]:
columns_drop_metmast = ['Min_Raindetection', 'Avg_Raindetection', 'Avg_Raindetection', 'Anemometer1_Freq', 'Anemometer1_Offset', 'Anemometer1_CorrGain', 'Anemometer1_CorrOffset', 'Anemometer2_Freq', 'Anemometer2_Offset', 'Anemometer2_CorrGain', 'Anemometer2_CorrOffset', 'DistanceAirPress', 'AirRessureSensorZeroOffset']

columns_drop_signals = ['Prod_LatestAvg_ActPwrGen2', 'Prod_LatestAvg_ReactPwrGen2']

In [26]:
turbine_list=['T01','T06','T07', 'T09','T11']

df_time_stamp=Ft.complete_time_df(Ft.time_df(),turbine_list)
df_time_stamp['Timestamp'] = pd.to_datetime(df_time_stamp['Timestamp'])


/Users/miguelamaro/code/miguelamaro4/Final_Project/wmillfailprev/notebooks/Funtion_tools.py:62: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  time_list.append(pd.datetime(ano,mes,dia)+pd.Timedelta(minutes=10)*i)


In [149]:
df_failures = pd.read_csv(r'../rawdata/wind-farm-1-failures-training.csv', sep=';')


In [124]:

df_failures['Timestamp'] = pd.to_datetime(df_failures['Timestamp']).dt.tz_convert(None)
#df_failures[df_failures['Turbine_ID']=='T07']


In [126]:
df_failures['Component'].value_counts()

GENERATOR            7
GENERATOR_BEARING    6
HYDRAULIC_GROUP      5
TRANSFORMER          3
GEARBOX              2
Name: Component, dtype: int64

In [75]:
df_failures['Timestamp_new'] = df_failures.apply(lambda x: x['Timestamp'] - datetime.timedelta(minutes=x['Timestamp'].minute % -10,
                             seconds=x['Timestamp'].second,
                             microseconds=x['Timestamp'].microsecond),axis=1)


#df_failures['Timestamp_new'] = pd.to_datetime(df_failures['Timestamp_new'])

In [76]:
print(type(df_failures['Timestamp_new'][0]))
print(type(df_failures['Timestamp'][0]))

print(type(df_time_stamp['Timestamp'][0]))



<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [111]:
df_failures_stamped = pd.merge(df_time_stamp, df_failures , how='left', left_on=['Turbine_ID','Timestamp'], right_on=['Turbine_ID','Timestamp_new'])

In [134]:
#df_failures_stamped[df_failures_stamped['Timestamp_x']=='2016-04-30 12:40:00']

In [114]:
components_list = ['GEARBOX', 'GENERATOR', 'GENERATOR_BEARING', 'TRANSFORMER', 'HYDRAULIC_GROUP']

In [115]:
for component in components_list:
    df_failures_stamped[f'Fail_{component}'] = df_failures_stamped.apply(lambda x: 1 if x['Component'] == component else 0, axis=1)

In [128]:
df_failures_final=df_failures_stamped[['Timestamp_x','Turbine_ID','Fail_GEARBOX','Fail_GENERATOR','Fail_GENERATOR_BEARING','Fail_TRANSFORMER','Fail_HYDRAULIC_GROUP']].rename(columns = {'Timestamp_x': 'Timestamp'}, inplace = False)


In [133]:
df_failures_final[df_failures_final['Fail_GEARBOX']==1]

,Timestamp,Turbine_ID,Fail_GEARBOX,Fail_GENERATOR,Fail_GENERATOR_BEARING,Fail_TRANSFORMER,Fail_HYDRAULIC_GROUP
28669,2016-07-18 02:10:00,T01,1,0,0,0,0
356308,2016-10-11 08:10:00,T09,1,0,0,0,0


In [153]:
df_signals = pd.read_csv('../rawdata/wind-farm-1-signals-training.csv', sep=';')#, nrows=100000)
df_signals = df_signals.drop(columns=columns_drop_signals)
#df_signals = df_signals[['Turbine_ID','Timestamp','Gen_RPM_Max']]
df_signals['Timestamp']=pd.to_datetime(df_signals['Timestamp'])
df_signals.head()

,Turbine_ID,Timestamp,Gen_RPM_Max,Gen_RPM_Min,Gen_RPM_Avg,Gen_RPM_Std,Gen_Bear_Temp_Avg,Gen_Phase1_Temp_Avg,Gen_Phase2_Temp_Avg,Gen_Phase3_Temp_Avg,...,Grd_Prod_PsbleInd_Avg,Grd_Prod_PsbleInd_Max,Grd_Prod_PsbleInd_Min,Grd_Prod_PsbleInd_Std,Grd_Prod_PsbleCap_Avg,Grd_Prod_PsbleCap_Max,Grd_Prod_PsbleCap_Min,Grd_Prod_PsbleCap_Std,Gen_Bear2_Temp_Avg,Nac_Direction_Avg
0,T11,2016-01-01 00:00:00+00:00,1339.4,1233.3,1270.9,18.8,42.0,59,61,60,...,-973.4,-752.6,-1000.0,56.5,973.4,1000.0,752.6,56.5,38,206.9
1,T06,2016-01-01 00:00:00+00:00,1270.0,1232.8,1248.5,6.8,42.0,51,52,53,...,-307.1,0.0,-880.6,180.0,307.1,880.6,0.0,180.0,35,204.6
2,T01,2016-01-01 00:00:00+00:00,1277.4,1226.1,1249.0,9.0,41.0,58,59,58,...,-144.4,0.0,-584.5,157.1,144.4,584.5,0.0,157.1,37,218.5
3,T09,2016-01-01 00:00:00+00:00,1376.7,1234.3,1272.0,24.7,40.0,58,57,58,...,-861.6,-224.5,-1000.0,185.5,861.6,1000.0,224.5,185.5,33,214.0
4,T07,2016-01-01 00:00:00+00:00,1317.5,1229.5,1254.9,13.8,41.0,62,62,61,...,-536.5,0.0,-1000.0,338.9,536.5,1000.0,0.0,338.9,39,197.3


In [68]:
df_signals['Timestamp'].dt.tz_convert('US/Central').dt.tz_convert(tz=None)

0        2016-01-01
1        2016-01-01
2        2016-01-01
3        2016-01-01
4        2016-01-01
            ...    
434140   2017-09-01
434141   2017-09-01
434142   2017-09-01
434143   2017-09-01
434144   2017-09-01
Name: Timestamp, Length: 434145, dtype: datetime64[ns]

In [137]:
import target as ta

In [150]:
df_target = ta.construct_df_target(df_failures)

/Users/miguelamaro/code/miguelamaro4/Final_Project/wmillfailprev/notebooks/Funtion_tools.py:62: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  time_list.append(pd.datetime(ano,mes,dia)+pd.Timedelta(minutes=10)*i)


In [ ]:
import target as ta
df_target = ta.construct_df_target(df_failures)

In [151]:
df_target

,Timestamp,Turbine_ID,Fail_GEARBOX,Fail_GENERATOR,Fail_GENERATOR_BEARING,Fail_TRANSFORMER,Fail_HYDRAULIC_GROUP
0,2016-01-01 00:00:00,T01,0,0,0,0,0
1,2016-01-01 00:10:00,T01,0,0,0,0,0
2,2016-01-01 00:20:00,T01,0,0,0,0,0
3,2016-01-01 00:30:00,T01,0,0,0,0,0
4,2016-01-01 00:40:00,T01,0,0,0,0,0
...,...,...,...,...,...,...,...
525600,2017-12-30 23:20:00,T11,0,0,0,0,0
525601,2017-12-30 23:30:00,T11,0,0,0,0,0
525602,2017-12-30 23:40:00,T11,0,0,0,0,0
525603,2017-12-30 23:50:00,T11,0,0,0,0,0


In [ ]:
df_signals['Timestamp'] = pd.to_datetime(df_signals['Timestamp']).dt.tz_convert(None)


In [159]:


df_signals_target = pd.merge(df_signals , df_target, left_on=['Turbine_ID','Timestamp'], right_on=['Turbine_ID','Timestamp'])

In [171]:
df_signals_target['Target_6C'] = df_signals_target.apply(lambda x: 1 if x['Fail_GEARBOX']==1  else (2 if  x['Fail_GENERATOR']==1 else (3 if x['Fail_GENERATOR_BEARING']==1 else (4 if x['Fail_TRANSFORMER']==1 else (5 if x['Fail_HYDRAULIC_GROUP']==1 else 0)))), axis=1)
                                      
                        
                        #x['Fail_GENERATOR_BEARING'] x['Fail_TRANSFORMER'] x['Fail_HYDRAULIC_GROUP']

In [172]:
df_signals_target

,Turbine_ID,Timestamp,Gen_RPM_Max,Gen_RPM_Min,Gen_RPM_Avg,Gen_RPM_Std,Gen_Bear_Temp_Avg,Gen_Phase1_Temp_Avg,Gen_Phase2_Temp_Avg,Gen_Phase3_Temp_Avg,...,Grd_Prod_PsbleCap_Min,Grd_Prod_PsbleCap_Std,Gen_Bear2_Temp_Avg,Nac_Direction_Avg,Fail_GEARBOX,Fail_GENERATOR,Fail_GENERATOR_BEARING,Fail_TRANSFORMER,Fail_HYDRAULIC_GROUP,Target_6C
0,T11,2016-01-01,1339.4,1233.3,1270.9,18.8,42.0,59,61,60,...,752.6,56.5,38,206.9,0,0,0,0,0,0
1,T06,2016-01-01,1270.0,1232.8,1248.5,6.8,42.0,51,52,53,...,0.0,180.0,35,204.6,0,0,0,0,0,0
2,T01,2016-01-01,1277.4,1226.1,1249.0,9.0,41.0,58,59,58,...,0.0,157.1,37,218.5,0,0,0,0,0,0
3,T09,2016-01-01,1376.7,1234.3,1272.0,24.7,40.0,58,57,58,...,224.5,185.5,33,214.0,0,0,0,0,0,0
4,T07,2016-01-01,1317.5,1229.5,1254.9,13.8,41.0,62,62,61,...,0.0,338.9,39,197.3,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434140,T11,2017-09-01,1266.3,1234.3,1250.6,5.1,46.0,60,61,61,...,524.4,109.1,43,96.8,0,0,0,0,0,0
434141,T09,2017-09-01,1268.4,1240.1,1251.9,6.0,44.0,62,61,62,...,340.3,121.6,40,105.0,0,0,0,0,0,0
434142,T06,2017-09-01,1284.2,1233.4,1252.4,8.8,49.0,63,64,65,...,171.8,272.2,45,84.1,0,0,0,0,0,0
434143,T01,2017-09-01,1299.0,1222.1,1255.3,12.4,45.0,64,64,64,...,314.2,203.6,44,109.5,0,0,0,0,0,0
